# サンプルコード　セマンティックセグメンテーション（U-Net）

In [ ]:
!git clone https://github.com/ARIM-Academy/Advanced_Tutorial_7-2.git
%cd Advanced_Tutorial_7-2

## 1.初期設定
### ライブラリインストール

In [ ]:
#汎用ライブラリ
import matplotlib.pyplot as plt

# モジュール
from model import *
from data import *

## 2. データセットの準備
microplasticsデータは「data/microplastics」フォルダに格納されており、これは二値分類タスクです。画像とマスクの入力形状は同じで、(バッチサイズ, 行数, 列数, チャネル = 1) という形になっています。

### データアーギュメンテーション（データ水増し）

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

myGene = trainGenerator(20,
                        'data/microplastics/train',
                        'image',
                        'label',
                        data_gen_args,
                        save_to_dir = "data/microplastics/train/aug"
                       )

### 合成データの作成
trainフォルダーには、imageファイルとmaskファイルがそれぞれ20枚あります。myGeneで定義されたデータアーギュメンテーションを3回まわすこと（num_batch = 3）で、それぞれ60枚の合成データを作成し、trainフォルダー配下の「aug」ファイルに格納します。

In [ ]:
num_batch = 3
for i,batch in enumerate(myGene):
    if(i >= num_batch):
        break

## npyファイルの作成

In [ ]:
train_image,train_mask = geneTrainNpy("data/microplastics/train/aug/","data/microplastics/train/aug/")
np.save("data/train_image.npy",train_image)
np.save("data/train_mask.npy",train_mask)

## 3.セマンティックセグメンテーションのモデル構築（U-Net）

In [ ]:
model = unet()

# ModelCheckpoint の設定（損失値に基づいて最良モデルを保存）
model_checkpoint = ModelCheckpoint('unet.keras', 
                                   monitor='loss', 
                                   verbose=1, 
                                   save_best_only=True)

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
   8/2000 ━━━━━━━━━━━━━━━━━━━━ 1:26:33 3s/step - accuracy: 0.6150 - loss: 0.6950

### 学習（パラメータ推定）

In [ ]:
%%time
history = model.fit(train_image,train_mask,
                    batch_size = 100,
                    epochs = 50,
                    verbose = 1,
                    validation_split = 0.2, 
                    shuffle = True,
                    callbacks=[model_checkpoint]
                   )

In [ ]:
#history = model.fit(myGene,
#                    steps_per_epoch=300,
#                    epochs=10,
#                    callbacks=[model_checkpoint]
#                    )

## 4.評価

In [ ]:
train_loss,train_acc = model.evaluate(train_image,train_mask)

print ('訓練データの損失関数の値', train_loss)
print ('訓練データの分類精度',train_acc)

### 結果出力

In [ ]:
#損失関数の値
loss = history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
#損失関数の値の可視化
plt.plot(loss, 'black', label='training')
plt.plot(val_loss, 'red', label='test')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#分類精度
acc = history.history['accuracy']
val_acc=history.history['val_accuracy']

In [ ]:
#分類精度の可視化
plt.plot(acc, 'black', label='training')
plt.plot(val_acc, 'red', label='test')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### 5. テストデータによる検証

In [ ]:
#model = unet()
#model.load_weights("unet.keras")

In [ ]:
testGene = testGenerator("data/microplastics/test/image/")

results = model.predict(testGene,
                        verbose=1)  # Remove 'steps' as you're providing the entire dataset

### 結果の保存

In [ ]:
saveResult("data/microplastics/test/image","data/microplastics/test/image/test_results",results)